In [193]:
import os
import re
import shutil #directory controls
import glob
import json
import sqlparse #new as of 20200629
from jsonpath_ng import jsonpath, parse
import datetime #new as of 20210411CS
import yaml
import fnmatch
import shutil
import errno
import subprocess

In [194]:
#set path variables

dbt_sources_file_path = "../dbt/square_dbt/models/sources/" #dbt-sources-path
dbt_models_file_path = "../dbt/square_dbt/models/" #dbt-sources-path
dbt_snapshots_file_path = "../dbt/square_dbt/snapshots/" #dbt-sources-path

dataform_output_sources_path = "_dataform_output/definitions/sources"
dataform_output_models_path = "_dataform_output/definitions"

dataform_root_path = "_dataform_output"




#20210411CS - added to use in the template creation dates with auto generated
dateYYYYMMDD = datetime.datetime.now().strftime("%Y%m%d") #.strftime("%Y%m%d-%H-%M-%S")

In [195]:
#find files on directories and subdirectories
def find_files(directory, pattern):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, pattern):
                filename = os.path.join(root, basename)
                yield filename

In [196]:
# list of all .yaml files in a directory
source_files = find_files(dbt_sources_file_path, '*.yml')

#YAML loader
def read_yaml_file(filename):
    with open(filename, 'r') as stream:
        try:
            yaml_dict = yaml.load(stream, Loader=yaml.FullLoader)
            return yaml_dict
        except yaml.YAMLError as exc:
            print(exc)

#YAML -> SQLx conerter
def create_sqlx_json_source_file(source_file_path):

#creating directory
    isExist = os.path.exists(dataform_output_sources_path)
    if not isExist:
        os.makedirs(dataform_output_sources_path)
    else:
        #delete all files from target
        shutil.rmtree(dataform_output_sources_path)
        os.makedirs(dataform_output_sources_path)
        
    for file in source_file_path:
        #reading all YAML files on source directory
        dic = read_yaml_file(file)
        #getting only sources entity from yaml file
        sources = dic["sources"]
        
        for source in sources:
            #declaring variables for each element of dictionary that will be used. tables = list, schema and database = str
            tables = source["tables"]
            database = source["database"]
            schema = source["schema"]
            #iterating through all tables for each schema
            for tables in tables:
                    #getting tables from list
                    tables= tables['name']
                    #creating json .sqlx structure
                    jmodel = "{\n"
                    jmodel += f'"type": "declaration",\n'                
                    jmodel += f'"database": "{database}",\n'
                    jmodel += f'"schema": "{schema}",\n'
                    jmodel += f'"name": "{tables}"'
                    jmodel += "\n}" #close out JSON file EOF

                    #parsing json and replacing double quotes for single quotes on keys
                    parsed = json.dumps(json.loads(jmodel), sort_keys=False, indent=2).replace('"type":','type:').replace('"database":','database:').replace('"schema":','schema:').replace('"name":','name:')

                    #creating .sqlx files
                    #write out to file in the appropriate location defined in variables
                    with open(f'{dataform_output_sources_path}/{schema}_{tables}.sqlx', "w") as jmodel_file:
                        #jmodel_file.write( script_header_template )
                        jmodel_file.write("config "+parsed)
                        jmodel_file.close()


In [197]:
def create_sqlx_models_files(model_files_path):
    for filename in model_files_path:  
        single_file_name = os.path.basename(filename)
        destination_path = (dataform_output_models_path+(filename[filename.find('models')+len('models'):filename.rfind('\\')]))+'/'
        destination_full_path = (destination_path+(os.path.basename(filename)).replace("sql","sqlx"))
        
        
        #print(destination_path)
        #print(single_file_name)
        #print(destination_full_path)
     #######copying files to new directory##############   
        try:
            #copy files if directory already exists
            
            shutil.copyfile(filename,destination_full_path)
        except IOError as e:
            # ENOENT(2): file does not exist, raised also on missing dest parent dir
            if e.errno != errno.ENOENT:
                raise
            # create directory if not exists
            
            os.makedirs(destination_path)
            # copy source files to new directory
            
            shutil.copyfile(filename,destination_full_path)
    #######end copying files to new directory##############

    #######reading file and replacing model syntax differences########
        # Read in the file
        with open(destination_full_path, 'r') as file :
            filedata = file.read()

            #getting header data to be replaced
            header=filedata[filedata.find("{{"):filedata.find("}}")+2]
            header_old=header

            header_replace_dict = {"{{":"", "}}":"}", "materialized":"type","=":":","(":"{",")":"","'":'"'}

            #iterate through dictionary keys
            for key in header_replace_dict.keys():
                #replacing all headers
                header = header.replace(key, header_replace_dict[key])         


        #writing file with replaced header and models references
        with open(destination_full_path, 'w') as file:
            #replacing files content
            new_file = filedata.replace(header_old,header)
            new_file = re.sub(r"[{][{][ ]{0,6}(ref|REF|Ref|SOURCE|source|Source)[ ]{0,9}[(][ ]{0,9}[']?",'${ref("', new_file)            
            new_file = re.sub(r"([ ]{0,9}[_'][.]{0,10}[ ]{0,20}(,)[.]{0,20}[ ]{0,20}[.]{0,20}['])",'_',new_file)
            new_file = re.sub(r"(['][)][ ]{0,9}[}])",'")',new_file)

            try:
                file.write(new_file)                
                print('Generated file: '+destination_full_path)
            except IOError as e:
                print ("I/O error({0}): {1}".format(e.errno, e.strerror))
    #######end reading file and replacing model syntax differences########
        
           
        



In [202]:
# list of all .yaml files in a directory
model_files = find_files(dbt_models_file_path, '*.sql')

def dbt_dataform_converter():
    
    #os.system("dataform init snowflake square_dataform")
    
    #deleting path
    shutil.rmtree(dataform_root_path)
    #creating source
    create_sqlx_json_source_file(source_files)
    #creating models
    create_sqlx_models_files(model_files)
    

In [203]:
dbt_dataform_converter()

Generated file: _dataform_output/definitions/master/W_CATALOG_ITEM_D.sqlx
Generated file: _dataform_output/definitions/master/W_CURRENCY_D.sqlx
Generated file: _dataform_output/definitions/master/W_CUSTOMERS_D.sqlx
Generated file: _dataform_output/definitions/master/W_DATE_D.sqlx
Generated file: _dataform_output/definitions/master/W_MERCHANT_LOCATION_D.sqlx
Generated file: _dataform_output/definitions/master/W_ORDERS_F.sqlx
Generated file: _dataform_output/definitions/master/W_PAYMENTS_F.sqlx
Generated file: _dataform_output/definitions/staging\CATALOG/V_CATALOG_CATEGORY_STG.sqlx
Generated file: _dataform_output/definitions/staging\CATALOG/V_CATALOG_DISCOUNT_STG.sqlx
Generated file: _dataform_output/definitions/staging\CATALOG/V_CATALOG_ITEM_MODIFIER_STG.sqlx
Generated file: _dataform_output/definitions/staging\CATALOG/V_CATALOG_ITEM_VARIATION_STG.sqlx
Generated file: _dataform_output/definitions/staging\CATALOG/V_CATALOG_TAX_STG.sqlx
Generated file: _dataform_output/definitions/stagin

In [218]:
#subprocess.call("dataform init snowflake square_dataform")
#shutil.copyfile("square_dataform",dataform_root_path)